In [1]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.layers import Flatten
from keras.layers import Embedding

C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

In [11]:
# Load data

def load_list(filename):
    vocabulary = []
    with open(filename, 'r') as f:
        for l in f:
            vocabulary.append(l.strip())
    return np.asarray(vocabulary)

def load_csv(filename):
    import csv
    
    sentence = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            sentence.append(row)
    return np.asarray(sentence).flatten()
    
pos_related = load_csv('./sentence_data/pos_related.csv')
neg_related = load_csv('./sentence_data/neg_related.csv')
pos_unrelated = load_csv('./sentence_data/pos_unrelated.csv')
neg_unrelated = load_csv('./sentence_data/neg_unrelated.csv')

In [12]:
print(pos_related.shape)
print(neg_related.shape)
print(pos_unrelated.shape)
print(neg_unrelated.shape)

(466,)
(198,)
(388,)
(149,)


In [13]:
# Stack the data

related_set = np.hstack((pos_related, neg_related))
print(related_set.shape)
y_related = np.ones(related_set.shape)
print(y_related.shape)

unrelated_set = np.hstack((pos_unrelated, neg_unrelated))
print(unrelated_set.shape)
y_unrelated = np.zeros(unrelated_set.shape)
print(y_unrelated.shape)

X_stack = np.hstack((related_set, unrelated_set))
y_stack = np.hstack((y_related, y_unrelated))

X_stack.shape

(664,)
(664,)
(537,)
(537,)


(1201,)

In [18]:
from sklearn.model_selection import train_test_split
X_train_corpus, X_test_corpus, y_train, y_test = train_test_split(X_stack, y_stack, test_size=0.33, random_state=42)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

token = r"(?u)\b[\w\'/]+\b"
tf_vectorizer = CountVectorizer(lowercase=True, max_df=1.0, min_df=1, binary=False, token_pattern=token)
tf_vectorizer.set_params(ngram_range=(1,1))

# X_vectorized = tf_vectorizer.fit_transform(X_stack)

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_stack, test_size=0.33, random_state=42)

tf_vectorizer.fit(X_train_corpus)

word_dict = tf_vectorizer.vocabulary_

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
word_dict

In [15]:
from textblob import TextBlob
import re

token_pattern = re.compile(token)
X = []
i=0
for sentence in X_stack:
    split = token_pattern.findall(sentence)
    seq = []
    for word in split:
        try:
            seq.append(word_dict[word])
        except KeyError:
            continue
    X.append(seq)
    
X = np.asarray(X)

print(X.shape)

NameError: name 'word_dict' is not defined

In [30]:
y_stack[260]

1.0

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_stack, test_size=0.33, random_state=42)

In [46]:
from keras.preprocessing.sequence import pad_sequences

print('Pad sequences (samples x time)')
maxlen=100
x_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (1248, 100)
x_test shape: (616, 100)


In [47]:
dict_len = len(tf_vectorizer.get_feature_names())
batch_size = 32
print(dict_len)

1806


In [48]:
hidden_neurons = 10

In [49]:
print('Build model...')
model = Sequential()
model.add(Embedding(dict_len, 500))
model.add(LSTM(output_dim=hidden_neurons, input_dim=500))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam')   

Build model...


C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(None, 500..., units=10)`
  after removing the cwd from sys.path.


In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 500)         903000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 10)                20440     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 923,451
Trainable params: 923,451
Non-trainable params: 0
_________________________________________________________________


In [51]:
from keras.callbacks import TensorBoard
from time import time

tensorboard = TensorBoard(log_dir="logs/{}".format(time()), write_graph=True, write_images=True)

In [52]:

print('Train...')
hist = model.fit(x_train, y_train, epochs=50, verbose=1, callbacks=[tensorboard])

Train...
Epoch 1/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6938
Epoch 2/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6928
Epoch 3/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6920
Epoch 4/50
1248/1248 [==============================] - 4s 4ms/step - loss: 0.6887
Epoch 5/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6797
Epoch 6/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6654
Epoch 7/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6549
Epoch 8/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6464
Epoch 9/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6393
Epoch 10/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6366
Epoch 11/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.6323
Epoch 12/50
1248/1248 [==============================] - 5s 4ms/step - loss:

In [268]:

y_predict = model.predict(x_test)

y = []

for pred in y_predict:
    if pred > 0.5:
        y.append(1)
    else:
        y.append(0)

        
np.sum(y_test == y)/len(y)

0.21915584415584416

In [12]:
# define the model
model = Sequential()
# model.add(Embedding(voc, 32, input_length=max_length))
# model.add(Flatten())
model.add(LSTM(output_dim=hidden_neurons, input_dim=in_neurons))

model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'], class_mode="binary")
# summarize the model
print(model.summary())

C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(None, 368..., units=10)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                147840    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 147,851
Trainable params: 147,851
Non-trainable params: 0
_________________________________________________________________
None
